In [14]:
import json
from glob import glob
from collections import OrderedDict

def trans(st):
    try:
        return int(st)
    except Exception:
        try:
            return float(st)
        except Exception:
            return st

def intersection(dicts):
    return dict(set.intersection(*(set(d.items()) for d in dicts)))

def flatten(conf_dict):
    new_dict = OrderedDict({})
    new_sub = OrderedDict({})
    for key, val in conf_dict.items():
        if not isinstance(val, dict):
            new_dict[key] = val
        else:
            new_sub[key] = val
    return new_dict, new_sub

exp_name = 'stanford_sri'

train_data = 'data_train_stanford_sri.txt'
test_data = 'data_test_stanford_sri.txt'

dirpath_base = '/Users/srivathsa/projects/SubtleGad'

# data_dict = {
#     'train': open('{}/data_lists/{}'.format(dirpath_base, train_data)).read().strip().split('\n'),
#     'test': open('{}/data_lists/{}'.format(dirpath_base, test_data)).read().strip().split('\n'),
# }

# with open('{}/experiments/{}/data.json'.format(dirpath_base, exp_name), 'w') as fpath_json:
#     st = json.dumps(data_dict, indent=2)
#     fpath_json.write(st)

In [2]:
base_dict = {
    "gpu": 2,
    "data_dir": "/raid/jon/data_full_tiantan/data",
    "learning_rate": 0.001,
    "max_data_sets": 100,
    "batch_size": 8,
    "num_epochs": 100,
    "queue_size": 4,
    "slices_per_input": 7,
    "shuffle": True,
    "file_ext": "h5",
    "val_split": 0.2,
    "verbose": 1,
    "log_dir": "/raid/jon/logs",
    "tb_dir": "/raid/jon/logs_tb",
    "hist_dir": "/raid/jon/history",
    "checkpoint_dir": "/raid/jon/checkpoints",
    "checkpoint_name": "mres_vgg_loss",
    "l1_lambda": 0.15,
    "ssim_lambda": 0.15,
    "no_save_best_only": True,
    "multiprocessing": 0,
    "num_workers": 4
}

In [54]:
flist = sorted(['train_tiantan_mpr_40data_20190626.sh', 'train_tiantan_ax_20190619.sh', 'train_tiantan_mpr_20190711.sh', 'train_tiantan_sag_20190619.sh', 'train_tiantan_mpr_20190626.sh', 'train_tiantan_mpr_2d_20190705.sh', 'train_tiantan_cor_20190619.sh', 'train_tiantan_20190612.sh'])

config_dicts = []

for fp in flist:
    fpath = '../scripts/{}'.format(fp)
    fstrs = open(fpath, 'r').read().strip().split(' ')[:-1]
    fdict = {l.split('=')[0].lower():trans(l.split('=')[1]) for l in fstrs}
    fdict['keystr'] = fp.replace('.sh', '').replace('train_tiantan_', '')
    config_dicts.append(fdict)

In [66]:
cdict = config_dicts[0]
exp = {**base_dict}
sub_exp = {**cdict}

for key, val in cdict.items():
    if key in exp:
        if exp[key] == val:
            del sub_exp[key]
        else:
            del exp[key]
keystr = sub_exp['keystr']
del sub_exp['keystr']
exp[keystr] = sub_exp

print(json.dumps(exp, indent=2))

{
  "data_dir": "/raid/jon/data_full_tiantan/data",
  "20190612": {
    "data_list": "data_lists/data_train_tiantan_20190612.txt",
    "l1_lambda": 0.6,
    "ssim_lambda": 0.4,
    "val_split": 0.1
  },
  "batch_size": 8,
  "num_epochs": 100,
  "max_data_sets": 100,
  "learning_rate": 0.001,
  "no_save_best_only": true,
  "file_ext": "h5",
  "checkpoint_name": "mres_vgg_loss",
  "num_workers": 4,
  "gpu": 2,
  "checkpoint_dir": "/raid/jon/checkpoints",
  "slices_per_input": 7,
  "log_dir": "/raid/jon/logs",
  "verbose": 1,
  "shuffle": true,
  "multiprocessing": 0,
  "hist_dir": "/raid/jon/history",
  "tb_dir": "/raid/jon/logs_tb",
  "queue_size": 4
}


In [18]:
for fpath_json in glob('/Users/srivathsa/projects/exp_tmp/*/config.json'):
    print(fpath_json)
    config_dict = json.loads(open(fpath_json, 'r').read())
    
    pre, pre_sub = flatten(config_dict['preprocess'])
    train, train_sub = flatten(config_dict['train'])
    inf, inf_sub = flatten(config_dict['inference'])
    common = []
    common.append(intersection([pre, train, inf]))
    common.append(intersection([pre, train]))
    common.append(intersection([pre, inf]))
    common.append(intersection([train, inf]))
    
    cdict = OrderedDict({k:v for d in common for k,v in d.items()})
    cdict['gpu'] = 0
    for key, val in cdict.items():
        if key in pre and pre[key] == val:
            del pre[key]
        
        if key in train and train[key] == val:
            del train[key]
        
        if key in inf and inf[key] == val:
            del inf[key]
        
        if 'gpu' in train:
            del train['gpu']
        if 'gpu' in inf:
            del inf['gpu']
        
    cdict['preprocess'] = OrderedDict(pre)
    for k, v in pre_sub.items():
        cdict['preprocess'][k] = v
        
    cdict['train'] = OrderedDict(train)
    for k, v in train_sub.items():
        cdict['train'][k] = v
    
    cdict['inference'] = OrderedDict(inf)
    for k, v in inf_sub.items():
        cdict['inference'][k] = v
    
    json_str = json.dumps(cdict, indent=2)
    outpath_json = fpath_json.replace('exp_tmp', 'SubtleGad/experiments')
    with open(outpath_json, 'w') as jsonf:
        jsonf.write(json_str)
    

/Users/srivathsa/projects/exp_tmp/super_model/config.json
/Users/srivathsa/projects/exp_tmp/hoag/config.json
/Users/srivathsa/projects/exp_tmp/tiantan/config.json
/Users/srivathsa/projects/exp_tmp/stanford_sri/config.json
/Users/srivathsa/projects/exp_tmp/tiantan_20190711/config.json
/Users/srivathsa/projects/exp_tmp/tiantan_sri/config.json
/Users/srivathsa/projects/exp_tmp/tiantan_gan/config.json
/Users/srivathsa/projects/exp_tmp/gen_siemens/config.json
/Users/srivathsa/projects/exp_tmp/sford_siemens/config.json
/Users/srivathsa/projects/exp_tmp/hoag_sri/config.json
/Users/srivathsa/projects/exp_tmp/tiantan_20190626/config.json
/Users/srivathsa/projects/exp_tmp/hoag_noscale/config.json
